# Install

Goal: At begining reduce and sequnce from n to b where b < n alllowing for savings of O(n^2-b^2) runtime

In [ ]:
!pip install huggingface_hub
!pip install -U spacy
!pip install  scipy
!python -m spacy download en_core_web_sm
!pip install datasets
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn
import torch.nn.functional as F
from huggingface_hub import login
from torch.utils.data import DataLoader
from datasets import load_dataset,  DatasetDict, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.auto import tqdm
from transformers import AutoTokenizer
import wandb
import os
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math
import time
login(token="ENTER TOKEN HERE")
wandb.login(key="ENTER KEY HERE")
device =  ('cuda' if torch.cuda.is_available() else 'cpu')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
class Language_Model(nn.Module):
  def __init__(self, base_llm_model, forget_layer, soft=False, choice_config=None ):
    super().__init__()
    self.base_llm_model = base_llm_model
    self.forget_layer = forget_layer
    self.soft=soft
    self.choice_config = choice_config

  def apply_mask(self, input_ids, mask, padding_value=0, attention =False):
        """
        Applies a forgetting mask to input_ids to retain only unmasked tokens and pads sequences to uniform length.

        Args:
            input_ids (torch.Tensor): Tensor of shape (batch_size, seq_len).
            mask (torch.Tensor): Binary mask of the same shape as input_ids,
                                where 1 indicates retained tokens and 0 indicates forgotten tokens.
            padding_value (int, optional): Value to pad the sequences to uniform length. Defaults to 0.

        Returns:
            torch.Tensor: Padded tensor of shape (batch_size, max_len), where max_len is the length of the longest retained sequence.
        """
        if not attention:
          if self.soft:
            embeddings = self.base_model.model.embed_tokens(input_ids)
            mask = mask.unsqueeze(-1).expand_as(embeddings)
            new_input_embeddings = mask*embeddings
            return {"embeddings": new_input_embeddings}

          #mask (batch , seq)
          if self.choice_config != None:
            if "percent" in self.choice_config: # top x percent
              percent = self.choice_config["percent"]
              k = int(mask.size(1) * percent)
              _, indices = torch.topk(mask, k, dim=1)
              new_mask = torch.zeros_like(mask)
              new_mask.scatter_(1, indices, 1)          # Set top-k values to 1
              mask = new_mask
            elif "threshold" in self.choice_config: #above threshold
              threshold = self.choice_config["threshold"]
              mask = (mask > threshold).float()
            count = torch.sum(mask == 0).item()

        updated_input_ids = []

        for i in range(input_ids.size(0)):
            # Use the mask to retain tokens from input_ids
            retained_input_ids = input_ids[i][mask[i].bool()]
            updated_input_ids.append(retained_input_ids)

        # Find the maximum sequence length after masking
        max_length = max(ids.size(0) for ids in updated_input_ids)

        if max_length == 0:
            #only return last tokens
          return input_ids[:, -1].rehsape(-1, 1)


          # Pad sequences to the maximum length
        padded_input_ids = torch.full(
              (input_ids.size(0), max_length),
              padding_value,
              dtype=input_ids.dtype,
              device=input_ids.device
          )

        for i, ids in enumerate(updated_input_ids):
            padded_input_ids[i, :ids.size(0)] = ids

        return {"input_ids": padded_input_ids}#, "count":count}


  def forward(self, input_ids, attention_mask=None, labels = None):
    mask = self.forget_layer(input_ids)
    data = self.apply_mask(input_ids, mask)
    if "input_ids" in data:
      input_ids = data["input_ids"]
      batch, seq_len = input_ids.size()
    else:
      input_embedings = data["embeddings"]
      batch, seq_len, _ = input_embedings.size()


    if attention_mask != None:
      attention_mask = self.apply_mask(attention_mask, mask)["input_ids"]

    if labels != None:
      labels = labels[:, -1*seq_len:]

    if "input_ids" in data:
      output = self.base_llm_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels )
    elif "embeddings" in data:
      output = self.base_llm_model(inputs_embeds=input_embedings, attention_mask=attention_mask, labels=labels )

    logits = output.logits  # Assuming 'output' is a named tuple or a dictionary

    if self.choice_config != None:

      percent = self.choice_config["percent"]
      k = int(mask.size(1) * percent)
      _, indices = torch.topk(mask, k, dim=1)
      new_mask = torch.zeros_like(mask)
      new_mask.scatter_(1, indices, 1)          # Set top-k values to 1
      mask = new_mask

    return_data = {"logits": logits, "seq_len": input_ids.size(1), "loss": output.loss, "mask":mask}
    if "count" in data:
      return_data["count"] = data["count"]
    return return_data


  def generate(self, input_ids, attention_mask =None, max_length=50):
      # Generate text by iteratively sampling next tokens
      masks = []
      for i in range(max_length):
          # Run input through the model to get next token probabilities
         # print("Mask", i)
          outputs = self.forward(input_ids,attention_mask = attention_mask)
          next_token_logits = outputs["logits"][:, -1, :]  # Only use the logits for the last token
          masks.append(outputs["mask"])

          # Sample the next token
          next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

          if attention_mask != None:
            attention_mask = torch.cat([attention_mask, torch.ones_like(next_token)], dim=-1)

          # Append the new token to the sequence
          input_ids = torch.cat([input_ids, next_token], dim=-1)

      return input_ids,masks

In [ ]:
class IdentityMask(nn.Module):
  def __init__(self):
      super(IdentityMask, self).__init__()

  def forward(self, input_ids):
      return torch.ones_like(input_ids, dtype=torch.bool)

In [ ]:
class RandomForgetMask(nn.Module):
  def __init__(self, forget_prob = .1):
      super(RandomForgetMask, self).__init__()
      self.forget_prob = forget_prob

  def create_random_forgetting_mask(self, input_ids):
      """
      Creates a mask for tokens to forget based on the forget rate.

      Args:
          input_ids (torch.Tensor): Tensor of shape (batch_size, seq_len).

      Returns:
          torch.Tensor: Binary mask of the same shape as input_ids,
                        where 1 indicates retained tokens and 0 indicates forgotten tokens.
      """
      batch_size, seq_len = input_ids.size()
      num_tokens_to_forget = int((seq_len) * self.forget_prob)
      masks = []

      for i in range(batch_size):
          # Generate random indices for tokens to forget
          forget_indices = torch.randperm(seq_len)[:num_tokens_to_forget]
          mask = torch.ones(seq_len, dtype=torch.bool, device=input_ids.device)
          mask[forget_indices] = False  # Mark forgotten tokens as 0
          masks.append(mask)

      return torch.stack(masks)

  def forward(self, input_ids):
      mask = self.create_random_forgetting_mask(input_ids)
      return mask


In [ ]:
class MLPForgetMask(nn.Module):
    def __init__(self, base_llm_model, hidden_dim=512):
        """
        Initializes the MLPForgetMask module.

        Args:
            base_llm_model (nn.Module): The base language model (e.g., a transformer model).
            tokenizer (PreTrainedTokenizer): The tokenizer corresponding to the language model.
            threshold (float): Threshold for forgetting. Probabilities below this are set to 0 (forgotten).
            hidden_dim (int): Hidden layer size for the MLP.
        """
        super(MLPForgetMask, self).__init__()
        self.embedding = base_llm_model.model.embed_tokens

        # Retrieve the embedding dimension from the base model
        embedding_dim = base_llm_model.config.hidden_size

        # Define the MLP
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Outputs a probability for retaining each token
        )

    def create_mlp_forgetting_mask(self, tokens):
        """
        Creates a mask for tokens to retain/forget using the MLP.

        Args:
            tokens (torch.Tensor): Token embeddings of shape (batch_size, seq_len, embedding_dim).

        Returns:
            torch.Tensor: Binary mask of shape (batch_size, seq_len),
                          where 1 indicates retained tokens and 0 indicates forgotten tokens.
        """
        batch_size, seq_len, embed_dim = tokens.size()

        # Flatten token embeddings for processing through MLP
        tokens_flat = tokens.view(-1, embed_dim)  # Shape: (batch_size * seq_len, embedding_dim)

        # Compute retain probabilities
        retain_probs = self.mlp(tokens_flat).view(batch_size, seq_len)  # Shape: (batch_size, seq_len)

        return retain_probs

    def forward(self, input_ids):
        """
        Generates the forgetting mask and applies it to the token embeddings.

        Args:
            input_ids (torch.Tensor): Input token IDs of shape (batch_size, seq_len).

        Returns:
            torch.Tensor: Masked token embeddings.
        """
        # Get token embeddings
        tokens = self.embedding(input_ids)  # Shape: (batch_size, seq_len, embedding_dim)

        # Create mask
        mask = self.create_mlp_forgetting_mask(tokens)  # Shape: (batch_size, seq_len)

        return mask


In [ ]:
class RNNForgetMask(nn.Module):
    def __init__(self, base_llm_model, threshold=0.5, hidden_dim=512, rnn_type="GRU", bidirectional=True):
        """
        Initializes the RNNForgetMask module.

        Args:
            base_llm_model (nn.Module): The base language model (e.g., a transformer model).
            threshold (float): Threshold for forgetting. Probabilities below this are set to 0 (forgotten).
            hidden_dim (int): Hidden layer size for the RNN.
            rnn_type (str): Type of RNN to use ("RNN", "LSTM", or "GRU").
        """
        super(RNNForgetMask, self).__init__()
        self.embedding = base_llm_model.model.embed_tokens
        self.threshold = threshold

        # Retrieve the embedding dimension from the base model
        embedding_dim = base_llm_model.config.hidden_size

        # Define the RNN layer
        rnn_class = {"RNN": nn.RNN, "LSTM": nn.LSTM, "GRU": nn.GRU}[rnn_type]
        self.rnn = rnn_class(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=bidirectional)

        if bidirectional:
            hidden_dim *= 2

        # Linear layer to output probabilities
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Outputs a probability for retaining each token
        )

    def create_rnn_forgetting_mask(self, tokens):
        """
        Creates a mask for tokens to retain/forget using the RNN.

        Args:
            tokens (torch.Tensor): Token embeddings of shape (batch_size, seq_len, embedding_dim).

        Returns:
            torch.Tensor: Binary mask of shape (batch_size, seq_len),
                          where 1 indicates retained tokens and 0 indicates forgotten tokens.
        """
        batch_size, seq_len, embed_dim = tokens.size()

        # Pass tokens through RNN
        rnn_output, _ = self.rnn(tokens)  # rnn_output: (batch_size, seq_len, hidden_dim)



        # Compute retain probabilities
        retain_probs = self.output_layer(rnn_output).squeeze(-1)  # Shape: (batch_size, seq_len)


        return retain_probs

    def forward(self, input_ids):
        """
        Generates the forgetting mask and applies it to the token embeddings.

        Args:
            input_ids (torch.Tensor): Input token IDs of shape (batch_size, seq_len).

        Returns:
            torch.Tensor: Masked token embeddings.
        """
        # Get token embeddings
        tokens = self.embedding(input_ids)  # Shape: (batch_size, seq_len, embedding_dim)

        # Create mask
        mask = self.create_rnn_forgetting_mask(tokens)  # Shape: (batch_size, seq_len)

        return mask

In [ ]:
def memory_check_and_empty():
    """Check GPU memory and clear cache only if necessary."""
    if torch.cuda.is_available():
        allocated_memory = torch.cuda.memory_allocated()
        reserved_memory = torch.cuda.memory_reserved()
        if reserved_memory - allocated_memory > 0.1 * reserved_memory:  # Threshold for unused memory
            torch.cuda.empty_cache()

In [ ]:
def load_model(name,device ):
  path = "/content/drive/My Drive/llama/models/" + name+"/"
  if device != "cuda":
    forget_layer = torch.load(path + "forget.pt", map_location=torch.device('cpu'))
  else:
    forget_layer = torch.load(path + "forget.pt")

  base_llm_model = AutoModelForCausalLM.from_pretrained(path).to(device)
  return base_llm_model, forget_layer

In [ ]:
#If using collab
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
EXAMPLES = 10
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
val_data = dataset["validation"].select(range(EXAMPLES))
prompts = [prompt for prompt in val_data["text"] if prompt.strip()]  # Filter out empty strings

In [ ]:
print(prompts[0])

 = Homarus gammarus = 



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
#baseline_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to(device)

In [ ]:
# Generate responses
import time
def generate_responses(prompts, model, tokenizer, max_length=50, custom= False, verbose = True):
    model.eval()
    responses = []
    masks = []
    start_time = time.time()
    for prompt in prompts:
        inputs = tokenizer([prompt], return_tensors="pt", truncation=True, max_length=max_length).to(device)
        with torch.no_grad():
            if custom:
              outputs, mask = model.generate(**inputs, max_length=max_length)
            else:
              outputs = model.generate(**inputs, max_new_tokens=max_length)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        responses.append(response)
        masks.append(mask)
    total_time = time.time() - start_time
    avg_time_per_response = total_time / len(prompts)
    return responses, masks, avg_time_per_response

In [ ]:
# Perplexity calculation
def calculate_perplexity(model, tokenizer, texts, batch_size=8, max_length=512, custom = False):
    model.eval()
    total_loss = 0
    total_tokens = 0
    notactive_tokens = 0
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(device)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            if custom:
              loss= outputs["loss"].item()
              if "count" in outputs:
                notactive_tokens += outputs["count"]
            else:
              loss = outputs.loss.item()
            total_loss += loss * inputs["input_ids"].numel()
            total_tokens += inputs["input_ids"].numel()
    ratio = (total_tokens -notactive_tokens)/total_tokens
    perplexity = math.exp(total_loss / total_tokens)
    return perplexity, ratio

In [ ]:
name = "ENTER NAME TO LOAD FOLDER"
base_llm_model, forget_layer = load_model(name, device)

<ipython-input-43-698f34e00cab>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  forget_layer = torch.load(path + "forget.pt", map_location=torch.device('cpu'))

A module th

AttributeError: _ARRAY_API not found

RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
initialization of _pywrap_checkpoint_reader raised unreported exception

In [ ]:
#Define how you want to chose values for mask, should be consistent with training for best results
#Percent will retain top X%, and treshold will retain all vluaes with score > X
#choice_config = None
#choice_config={"percent":.90}
#choice_config= {"threshold":.5}

In [ ]:
fine_tuned_model = Language_Model(base_llm_model, forget_layer, soft=False, choice_config=choice_config).to(device).eval()

In [ ]:
fine_tuned_responses, fine_tuned_time = generate_responses(prompts, fine_tuned_model, tokenizer, custom= True)

In [ ]:
fine_tuned_perplexity, fine_tuned_ratio = calculate_perplexity(fine_tuned_model, tokenizer, prompts, custom=True)

In [ ]:
# Generate responses
import time
def generate_response(prompt, model, tokenizer, max_length=10):
    model.eval()
    start_time = time.time()
    inputs = tokenizer([prompt], return_tensors="pt").to(device)
    with torch.no_grad():
        outputs, masks  = model.generate(**inputs, max_length=max_length)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(max_length)
    print("Response:", response[-1*max_length: ] )
    total_time = time.time() - start_time
    return response, masks

In [ ]:
p  = prompts[1][:len(prompts[1])//2]

In [ ]:
print(p)

In [ ]:
prompt = "enter prompt here"
response, masks= generate_response(p, fine_tuned_model, tokenizer, max_length=20) #

In [ ]:
!pip uninstall scipy numpy
!pip install numpy==1.26.0
!pip install scipy==1.13.3

In [ ]:
!pip install rouge_score

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def calculate_bleu(references, hypotheses):
    """
    Calculate BLEU score for a set of references and hypotheses.

    Args:
        references (list of str): Ground truth text.
        hypotheses (list of str): Generated text by the model.

    Returns:
        float: Average BLEU score.
    """
    smoothing_function = SmoothingFunction().method1
    bleu_scores = []

    for ref, hyp in zip(references, hypotheses):
        ref_tokens = ref.split()
        hyp_tokens = hyp.split()
        score = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothing_function)
        bleu_scores.append(score)

    return sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0.0

def calculate_rouge(references, hypotheses):
    """
    Calculate ROUGE scores for a set of references and hypotheses.

    Args:
        references (list of str): Ground truth text.
        hypotheses (list of str): Generated text by the model.

    Returns:
        dict: Average ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for ref, hyp in zip(references, hypotheses):
        scores = scorer.score(ref, hyp)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    return {key: sum(values) / len(values) if values else 0.0 for key, values in rouge_scores.items()}


In [ ]:
# Calculate BLEU and ROUGE
fine_tuned_bleu = calculate_bleu(prompts, fine_tuned_responses)
fine_tuned_rouge = calculate_rouge(prompts, fine_tuned_responses)

In [ ]:
print(f"Fine-Tuned Model Perplexity: {fine_tuned_perplexity}")
print(f"Fine-Tuned Model Ratio: {fine_tuned_ratio}")
print(f"Fine-Tuned Avg Inference Time: {fine_tuned_time:.4f} seconds per response")
print(f"Fine-Tuned Model BLEU: {fine_tuned_bleu}")
print(f"Fine-Tuned Model ROUGE: {fine_tuned_rouge}")

In [ ]:
# Log results to WandB
wandb.init(
    project="llama_eval",  # Replace with your WandB project name
    config={
        "description": "Evaluation of fine-tuned vs baseline language models",
    }
)
logs = {
    "name": name,
    "Fine-Tuned Model Perplexity": fine_tuned_perplexity,
    "Fine-Tuned Avg Inference Time (s)": fine_tuned_time,
    "Fine-Tuned Model BLEU": fine_tuned_bleu,
    "Fine-Tuned Model ROUGE": fine_tuned_rouge,
}
if choice_config != None:
  logs.update(choice_config)
wandb.log(logs)